In [1]:
from __future__ import division, absolute_import, print_function

import sys
import os
import pickle
import numpy as np
import random

import h5py
from collections import Counter
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from imblearn import over_sampling 
from tables import *
from sklearn.cluster import MiniBatchKMeans
import time
from sklearn import preprocessing

#import keras

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)


from src.imbalance_functions import *
from src.Target import Target

os.environ['PYTHONHASHSEED'] = '0' 
np.random.seed(8)
random.seed(8)
#tf.random.set_seed(8)

Using TensorFlow backend.


Para el semi-supervised voy a usar la division de training, validation, test de data_no_resampled

In [2]:
nfolds = 10
nclusters = 100
batch_size = 1000
protein_type = "GPCRs" #"kinases"

In [3]:
#cargamos fingeprints
file_fps = "/home/angela3/imbalance_pcm/raw_data/dcid_fingerprint.tsv"
fps_df = pd.read_csv(file_fps, sep="\t")

In [4]:
# Loading activities
activity_file = "".join((absPath, "data/", protein_type, "_activity.csv"))
activity_df = pd.read_csv(activity_file, sep="\t")
print(activity_df.info())
print(activity_df.head())                  
                       
activity_df["comp_prot"] = activity_df.apply(prot_comp_composite, axis=1) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200523 entries, 0 to 200522
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                200523 non-null  int64  
 1   DeepAffinity Protein ID   200523 non-null  object 
 2   Uniprot ID                200523 non-null  object 
 3   DeepAffinity Compound ID  200523 non-null  object 
 4   CID                       200523 non-null  int64  
 5   activity                  200523 non-null  float64
 6   label                     200523 non-null  float64
 7   Canonical SMILE           200523 non-null  object 
 8   Sequence                  200523 non-null  object 
 9   family                    200523 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 15.3+ MB
None
   Unnamed: 0 DeepAffinity Protein ID Uniprot ID DeepAffinity Compound ID  \
0          32                    UYMO     Q92847                     4vb

In [5]:
class Protein_Compound_Complex(IsDescription):
    #CID = UInt16Col()
    da_comp_id = StringCol(4) 
    da_prot_id = StringCol(4)
    #uniprot_id = StringCol(6)
    #activity = Float16Col()
    label = UInt16Col()
    #canonical_smiles = StringCol(100)
    sequence = StringCol(2000)
    #prot_family = StringCol(5)
    #comp_cluster = UInt16Col()
    fingerprint = StringCol(900)
    splitting_0 = UInt8Col()
    splitting_1 = UInt8Col()
    splitting_2 = UInt8Col()
    splitting_3 = UInt8Col()
    splitting_4 = UInt8Col()
    splitting_5 = UInt8Col()
    splitting_6 = UInt8Col()
    splitting_7 = UInt8Col()
    splitting_8 = UInt8Col()
    splitting_9 = UInt8Col()

In [6]:
#Loading maximum lengths of proteins and compounds
with open("".join((absPath, 'data/prot_max_len.pickle')), "rb") as input_file:
    max_len_prot = pickle.load(input_file)
#Defining protein dictionary    
instarget = Target("AAA")
prot_dict = instarget.predefining_dict()  

In [8]:
for fold in range(nfolds):
    print("Fold:", fold)
    file_list = "".join((absPath, "data/", protein_type, "/no_resampling/splitting_lists/splitting_", str(fold),"_list.pickle"))
    with open(file_list, "rb") as input_file:
        splitting_list = pickle.load(input_file)
        
    #Opening HDF5 with data
    file_h5 = "".join((absPath, "data/", protein_type, "/no_resampling/compounds_activity.h5"))
    f = h5py.File(file_h5, 'r')
    group = '/activity'
    table = "prot_comp"
    
    print("TEST")
    #test set
    splitting_list[2].sort()
    #Loading test set data because we will remove it from the activity data
    comp_ids_list = list(f[group][table][splitting_list[2]]["da_comp_id"])
    prot_ids_list = list(f[group][table][splitting_list[2]]["da_prot_id"])
    print("There are ", len(comp_ids_list), " sample in the test set")
    
    comp_prot_list = [(comp_ids_list[i].decode("utf-8")+","+prot_ids_list[i].decode("utf-8")) for i in range(len(comp_ids_list))]
                  
    # Removing rows whose composite prot-comp is in the test list
    activity_without_test = activity_df[~activity_df.comp_prot.isin(comp_prot_list)]
    
    activity_test = activity_df[activity_df.comp_prot.isin(comp_prot_list)]
    activity_test.drop(["Unnamed: 0"], axis=1, inplace=True)
    
    activity_test_fps = pd.merge(activity_test, fps_df, "left", on=["DeepAffinity Compound ID"])
    
    if not os.path.exists("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                       str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                 str(fold), "/")))  
        
    activity_test_fps.to_csv("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                                      "/activity_data.csv")))
    #open a HDF5 file with write options
    file_h5 = open_file("".join((absPath, "data/",protein_type, "/semi_resampling/", str(fold),
                                 "/compounds_activity_test.h5")), "w")
    root = file_h5.root
    group = file_h5.create_group(root, "activity")
    table = file_h5.create_table('/activity', "prot_comp", Protein_Compound_Complex) 
    pair = table.row

    for index,row in activity_test_fps.iterrows():
        #pair["CID"] = row["CID"]
        pair["da_comp_id"] = row["DeepAffinity Compound ID"]
        pair["da_prot_id"] = row["DeepAffinity Protein ID"]
        #pair["activity"] = row["activity"]
        pair["label"] = row["label"]
        #pair["canonical_smiles"] = row["Canonical SMILE"]
        pair["sequence"] = row["Sequence"]
        #pair["prot_family"] = row["family"]
        #pair["comp_cluster"] = row["cluster_label"]
        pair["fingerprint"] = row["Fingerprint Feature"]
        pair.append()
    table.flush()
    file_h5.close()
    
    print("SMOTEing the rest of the data")
    # Now I should SMOTE the rest of the data
    #Loading list of unique proteins (only those were SMOTE could be applied)
    #with open("".join((absPath, "data/", protein_type, "/smote_prots.pickle")), 'rb') as handle:
    with open("".join((absPath, "data/", protein_type, "/GPCRs_prots.pickle")), "rb") as handle:
        unique_prots = pickle.load(handle)
    
    sub_df = activity_without_test[activity_without_test["DeepAffinity Protein ID"].isin(unique_prots)]
                                 
    if not os.path.exists("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                       str(fold), "/resampled_data/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                 str(fold), "/resampled_data/")))
    if not os.path.exists("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                       str(fold), "/subdataframes/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                 str(fold), "/subdataframes/")))                                 
    print("Entering into loop")
    for prot in unique_prots:
        sub_prot = sub_df[sub_df["DeepAffinity Protein ID"] == prot]
        prot_fps = pd.merge(sub_prot[["DeepAffinity Protein ID", "DeepAffinity Compound ID", "label", 
                                  "Canonical SMILE", "Sequence", "family"]], 
                        fps_df, "left", on=["DeepAffinity Compound ID"])
        
        file_subdf = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                              "/subdataframes/", prot, ".csv"))
        prot_fps.to_csv(file_subdf)
                                 
    print("Entering into second loop")
    prots_que_pasan_el_smote = []

    for prot in unique_prots:
        print(prot)
        sm = over_sampling.SMOTE(sampling_strategy=1.0, random_state=42)
    
        file_subdf = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                              "/subdataframes/", prot, ".csv"))
        sub_prot = pd.read_csv(file_subdf)
    
        sub_prot["separated_fps"] = sub_prot["Fingerprint Feature"].apply(separating_fps)
        X = pd.DataFrame.from_dict(dict(zip(sub_prot["separated_fps"].index, sub_prot["separated_fps"].values))).transpose()
        Y = sub_prot["label"].values
        try:
            Xmot, Ymot = sm.fit_resample(X, Y)
            print(Xmot.shape)
            prots_que_pasan_el_smote.append(prot)
        except:
            print(" The specified ratio required to remove samples from the minority class while trying to generate new samples.")
            continue
        pickle_path = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                               "/resampled_data/", prot, ".pickle"))
        with open(pickle_path, 'wb') as handle:
            pickle.dump((Xmot, Ymot), handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Hay", len(prots_que_pasan_el_smote), "proteinas que cumplen las condiciones para el SMOTE")
      
    print("Entering into third loop")
    df_lists = []
    for prot in prots_que_pasan_el_smote:
        print(prot)
        #Load data pickle
        pickle_path = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold),
                               "/resampled_data/", prot, ".pickle"))
        with open(pickle_path, 'rb') as handle:
            Xmot, Ymot = pickle.load(handle)
        df = Xmot.copy()
        df["Y"] = Ymot
        print(len(Ymot))
        #df.drop_duplicates(subset=Xmot.columns, keep="first", ignore_index=True, inplace=True)
        df["prot"] = prot
        df_lists.append(df) 

    resampled_activity = pd.concat(df_lists)
    resampled_activity["fingerprint"] = resampled_activity[[i for i in range(881)]].apply(lambda row: "".join(row.values.astype(str)), axis=1)
    unique_compounds_df = resampled_activity.drop_duplicates(subset="fingerprint", keep="first", ignore_index=True)
    
    names_comps = ["c"+str(i) for i in range(unique_compounds_df.shape[0])]
    unique_compounds_df["comp_ID"] = names_comps

    unique_compounds_df.to_csv("".join((absPath, "data/", protein_type, "/semi_resampling/", 
                                        str(fold), "/unique_compounds_balanced_fps.csv")))
    resampled_activity_compID = pd.merge(resampled_activity, unique_compounds_df[["fingerprint", "comp_ID"]],
                                     "left", on=["fingerprint"])
    ### CLUSTERING
    print("CLUSTERING")
    #keeping only minimum info for clustering
    only_comps = unique_compounds_df.drop(["prot", "Y"], axis=1)
    only_comps.drop_duplicates("comp_ID", inplace=True)
    only_comps.reset_index(inplace=True)
    only_comps.info()
                                 
    sample_indices = np.arange(0, only_comps.shape[0])
    sample_indices = np.random.permutation(sample_indices)
    generate_batches = batch_generator(batch_size, only_comps, sample_indices)
    
    #K-Means
    model = MiniBatchKMeans(n_clusters=nclusters, init='k-means++', compute_labels=True)
    sse = {}
    labels = []
    comp_ids = []
    clusters_centers = {}
    #i=0
    for i, batch in enumerate(generate_batches):
        print("Iteration ", i)
        t0 = time.time()
        df = pd.DataFrame(batch)
        print(df["comp_ID"])
        comp_ids.append(df["comp_ID"])
        to_array = df.drop(['comp_ID'], axis=1).values
        to_array = preprocessing.scale(to_array)
    
        model.partial_fit(to_array)
        print("The inertia for the batch %s is %s" % (i, model.inertia_))
        t_mini_batch = time.time() - t0
        print(t_mini_batch)
        sse[i] = model.inertia_
        labels.append(model.labels_)
        clusters_centers[i] = model.cluster_centers_
    labels_array = np.hstack(labels)
    comp_ids_list = np.hstack(comp_ids)

    #Joining compound IDs and cluster labels
    compound_clusters = pd.DataFrame({'comp_ID':comp_ids_list,
                                  'cluster_label':labels_array})

    compound_clusters.to_csv("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                                      "/compound_clusters.csv")), header=True)
    #merging the resampled activity dataframe with the clusters
    activity_with_clusters = pd.merge(resampled_activity_compID, compound_clusters, on=["comp_ID"], how="left")
    activity_with_clusters.info()
    #the resampled activity cluster does not have sequence info!!!
    prots_info = activity_df[["DeepAffinity Protein ID", "Sequence"]].drop_duplicates()
    prots_info.info()                             
    activity_with_clusters.rename(columns={"prot": "DeepAffinity Protein ID"}, inplace=True)       
    activity_resampled_clustered = pd.merge(activity_with_clusters[["DeepAffinity Protein ID", "comp_ID", "fingerprint", "Y", "cluster_label"]], 
                                        prots_info, "left", on=["DeepAffinity Protein ID"])
    activity_resampled_clustered.to_csv("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), "/activity_clusters.csv")), 
                              sep="\t", header=True)
    
    print("SPLITTING TRAINING-VALIDATION")
    #splitting training/validation
    compounds_classif = accumulated_size_clusters(activity_resampled_clustered)
    compounds_classif = training_test_split_semi(compounds_classif, 90, 10, 0, fold)
    #Joining smiles-label dataframe with label information dataframe
    name_column = "splitting_" + str(fold)
    activity_resampled_clustered = pd.merge(compounds_classif[["cluster_label", name_column]], 
                                 activity_resampled_clustered, on="cluster_label")
    print(activity_resampled_clustered[name_column].value_counts())
    print("PRINTING ACTIVITY_RESAMPLED_CLUSTERED INFO")
    print(activity_resampled_clustered.info())
    
    #count_split = activity_resampled_clustered[[name_column, "Y"]].groupby([name_column, 
    #                                      "Y"]).size().unstack(fill_value=0)
    #count_split_df = pd.DataFrame(count_split)
    #count_split_df_melt = pd.melt(count_split_df.reset_index(), id_vars = name_column, value_vars=[0.0,1.0])
    #count_split_df_melt["splitting_fold"] = name_column
    #count_split_df_melt = count_split_df_melt.rename(columns = {name_column:"split_set"})
    #print(count_split)
    #count_list.append(count_split_df_melt)
                                                
    #SAVING DATA
    print("SAVING DATA")
    #open a HDF5 file with write options
    file_h5 = open_file("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold),
                                 "/compounds_activity.h5")), "w")
    root = file_h5.root
    group = file_h5.create_group(root, "activity")
    table = file_h5.create_table('/activity', "prot_comp", Protein_Compound_Complex) 
    pair = table.row

    for index,row in activity_resampled_clustered.iterrows():
        #pair["CID"] = row["CID"]
        pair["da_comp_id"] = row["comp_ID"]
        pair["da_prot_id"] = row["DeepAffinity Protein ID"]
        #pair["activity"] = row["activity"]
        pair["label"] = row["Y"]
        #pair["canonical_smiles"] = row["Canonical SMILE"]
        pair["sequence"] = row["Sequence"]
        #pair["prot_family"] = row["family"]
        #pair["comp_cluster"] = row["cluster_label"]
        pair["fingerprint"] = row["fingerprint"]
        pair[name_column] = row[name_column]
        pair.append()
    table.flush()
    file_h5.close()

Fold: 0
TEST
There are  21135  sample in the test set
SMOTEing the rest of the data
Entering into loop
Entering into second loop
UYMO
(1638, 881)
0BJE
(464, 881)
78TA
(176, 881)
RJYX
(112, 881)
95E2
(3494, 881)
P59Q
(934, 881)
XY2A
(444, 881)
3Y9A
(1076, 881)
J6QF
(1740, 881)
I8AL
(4786, 881)
K1O5
(5586, 881)
Q6RX
(8344, 881)
L3MG
(208, 881)
5W5R
(4856, 881)
3KD1
(208, 881)
MUMW
(438, 881)
SH36
(316, 881)
2CH1
(672, 881)
TOAR
(844, 881)
F5KQ
(1792, 881)
N0SR
(266, 881)
230E
(440, 881)
OYY9
(742, 881)
7WTN
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
8VB1
(164, 881)
CVFR
(152, 881)
LETU
(914, 881)
LFAB
(84, 881)
C0XK
(248, 881)
DFO0
(4686, 881)
1ZXI
(282, 881)
SXV6
(6676, 881)
DPHN
(3222, 881)
ZALS
(412, 881)
Y9QF
(4204, 881)
YO6L
(4918, 881)
3Y7S
(252, 881)
5GL1
(172, 881)
NXHT
(238, 881)
JVOZ
(38, 881)
4C54
(232, 881)
06WU
(208, 881)
662N
(1678, 881)
ES23
(2814, 881)
KPOO
 The specified ratio required to remove samples f

 The specified ratio required to remove samples from the minority class while trying to generate new samples.
7UYC
(1242, 881)
0YVO
(938, 881)
43BP
(654, 881)
J4XL
(394, 881)
GKL5
(36, 881)
EN75
(124, 881)
4O6Q
(2248, 881)
CNG4
(32, 881)
79GX
(86, 881)
NJJH
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
8TEG
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
2B66
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
HGTX
(40, 881)
NWNZ
(342, 881)
6YX0
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
4SYY
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
MDPN
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
B739
(742, 881)
BQC9
(94, 

(1674, 881)
93M7
(214, 881)
W166
(368, 881)
H878
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
NG45
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
MCUD
(184, 881)
S77L
(624, 881)
KGJK
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
TUIG
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
5QYK
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
N1SI
(42, 881)
CZLO
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
VX6K
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
EV5K
 The specified ratio required to remove samples from the minority class while trying to generate ne

(360, 881)
V40B
(18, 881)
PRD3
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
BFIR
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
DF2W
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
7ESP
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
4MJ3
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
M2GJ
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
EBWZ
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
OBOK
 The specified ratio required to remove samples from the minority class while trying to generate new samples.
TDSB
 The specified ratio required to remove samples f

/home/angela3/venv/lib/python3.6/site-packages/ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

CLUSTERING
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120913 entries, 0 to 120912
Columns: 884 entries, index to comp_ID
dtypes: int64(882), object(2)
memory usage: 815.5+ MB
Iteration  0
0       c28562
1       c61414
2       c81205
3       c35031
4        c5439
        ...   
995     c60703
996     c88161
997     c52313
998     c70599
999    c118066
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 0 is 292221.0946093806
0.6623563766479492
Iteration  1
0       c54257
1       c63440
2       c73915
3       c72478
4      c115759
        ...   
995     c28026
996    c112522
997     c12821
998     c33104
999     c97973
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 1 is 362755.53139282647
0.3586604595184326
Iteration  2
0       c30670
1       c39292
2       c64495
3       c51181
4       c32981
        ...   
995    c113487
996     c56325
997     c60580
998     c18115
999     c69540
Name: comp_ID, Length: 1000, dtype: object
The inertia for th

Iteration  28
0       c62810
1       c56786
2      c110537
3       c55834
4       c37732
        ...   
995    c119385
996     c56405
997     c64219
998      c6060
999     c58440
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 28 is 372947.14653457893
0.3201484680175781
Iteration  29
0      c71253
1      c82204
2      c25454
3       c5162
4      c58198
        ...  
995    c24610
996     c9945
997    c49499
998    c31291
999    c54261
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 29 is 352905.88446176914
0.3487992286682129
Iteration  30
0       c96020
1      c105868
2      c111946
3       c16222
4       c88225
        ...   
995     c42607
996    c109365
997    c105558
998    c111119
999     c82831
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 30 is 360963.0318140884
0.3312647342681885
Iteration  31
0       c7597
1      c29163
2      c56310
3      c53411
4      c45685
        ...  
995    c28875
996    c46112
997    c99

Iteration  57
0       c12991
1        c1972
2      c110293
3       c22527
4       c87563
        ...   
995     c35910
996    c105201
997     c92545
998     c21242
999     c59132
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 57 is 355468.9043612415
0.318803071975708
Iteration  58
0       c50521
1       c41805
2       c60419
3       c26711
4      c107786
        ...   
995     c43610
996     c80704
997     c45903
998    c113490
999     c17571
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 58 is 346880.5256881436
0.3369781970977783
Iteration  59
0        c4009
1       c88346
2       c73402
3       c12801
4       c73808
        ...   
995     c47214
996     c94363
997     c11422
998    c114313
999    c105142
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 59 is 360814.2683682149
0.32941365242004395
Iteration  60
0      c17096
1       c5221
2      c63100
3       c7238
4      c13888
        ...  
995      c334
996    c84933
9

Iteration  86
0       c80465
1       c34060
2      c119866
3       c63990
4       c61401
        ...   
995     c74193
996     c82051
997     c49691
998     c95589
999     c58165
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 86 is 370064.4774352971
0.32750797271728516
Iteration  87
0      c119948
1      c120271
2       c63273
3       c45583
4       c11811
        ...   
995    c120226
996      c4445
997     c77346
998      c9380
999    c102318
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 87 is 358526.1961736545
0.3286011219024658
Iteration  88
0      c103669
1        c4294
2       c87995
3       c95105
4      c118209
        ...   
995     c26168
996     c12913
997     c89730
998     c34401
999     c44588
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 88 is 363541.96521308884
0.3413064479827881
Iteration  89
0       c46166
1       c49744
2       c72927
3       c59966
4       c60929
        ...   
995     c13763
996   

Iteration  115
0       c88071
1       c99604
2      c116556
3       c66407
4        c8988
        ...   
995     c14896
996     c37107
997     c80512
998     c55191
999     c41139
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 115 is 357632.30301184097
0.3812851905822754
Iteration  116
0       c26709
1      c101383
2       c74075
3       c45337
4       c55669
        ...   
995     c93565
996     c44527
997     c78782
998    c119970
999     c31672
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 116 is 367029.27275375865
0.35042405128479004
Iteration  117
0      c15630
1      c24036
2      c52102
3      c59705
4      c55494
        ...  
995     c5061
996     c5146
997    c77797
998    c53114
999    c47363
Name: comp_ID, Length: 1000, dtype: object
The inertia for the batch 117 is 351477.4865692928
0.3382418155670166
Iteration  118
0      c107670
1        c3150
2       c30193
3       c73563
4       c34382
        ...   
995     c42571
996     c

In [ ]:
#count_list = []
#for i in range(nfolds):
#    name_column = "splitting_" + str(i)
#    count_split = activity_resampled_clustered[[name_column, "Y"]].groupby([name_column, 
#                                          "Y"]).size().unstack(fill_value=0)
#    count_split_df = pd.DataFrame(count_split)
#    count_split_df_melt = pd.melt(count_split_df.reset_index(), id_vars = name_column, value_vars=[0.0,1.0])
#    count_split_df_melt["splitting_fold"] = name_column
#    count_split_df_melt = count_split_df_melt.rename(columns = {name_column:"split_set"})
#    print(count_split)
#    count_list.append(count_split_df_melt)

In [12]:

    
for fold in range(nfolds):
    file_h5 = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), "/compounds_activity.h5"))
    f = h5py.File(file_h5, 'r')
    group = '/activity'
    table = "prot_comp"
    n_samples = len(f[group][table])
    sample_indices = np.arange(0, n_samples)
    sample_indices = np.random.permutation(sample_indices)

    if not os.path.exists("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                                   "/splitting_lists/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                             "/splitting_lists/")))
    #Opening HDF5 with data
    file_h5 = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), "/compounds_activity.h5"))
    f = h5py.File(file_h5, 'r')
    group = '/activity'
    table = "prot_comp"
    column_name = "splitting_" + str(fold)
    training_list, validation_list = splitting_division_semi(f, group, 
                                                                   table, 
                                                                   sample_indices, 
                                                                   column_name)
    
    pickle_filename = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                               "/splitting_lists/", column_name, "_list.pickle"))
    with open(pickle_filename, "wb") as handle:
        pickle.dump((training_list, validation_list), handle)
    f.close()

322.046373128891
321.2069020271301
323.30576515197754
321.4697902202606
323.14859342575073
330.49700355529785
321.84386587142944
338.2929902076721
319.3059275150299
321.9341480731964


In [11]:
pickle_filename = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                               "/splitting_lists/", column_name, "_list.pickle"))

#with open(pickle_filename, "rb") as handle:
#        prueba = pickle.dump((training_list, validation_list), handle)